In [30]:
import sync_wandb_run
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import utils as pu
from plot import create_plot_with_values
from plot_pretty import create_ultimate_plotly_plot, generate_scatter_plot, generate_scatter_plot_accuracy, generate_scatter_plot_gridscore
import plotly_utils

In [31]:
entity = "raphaelmaser"  # Your W&B username or team
project = "Improved-Interpretability-and-Concepts"  # Your W&B project name
df = sync_wandb_run.load_wandb_runs()
df.count()  # Display the number of runs loaded
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

val_accuracy = "Validation Accuracy"
grid_score = "PG Score"
frozen_layers = "config_define_frozen_layers"
trainable_params = "Trainable Parameters (M)"
time="Runtime (h)"
figsize=(10,4)


# Comparison between HugginFace PEFT implementations

Adapters only applied to qkv projection without ReLU!


In [32]:

vitc = df[df['group'].isin(["vitc-peft-randlora-ranks", "vitc-peft-lora-ranks", "vitc-peft-dora-ranks", "vitc-peft-shira-ranks", "vitc-peft-lora-eva-ranks", "vitc-peft-dora-eva-ranks"])] 
vitc["config_weight_decay"] = vitc["config_weight_decay"].astype(float)  # Ensure weight decay is float type
vitc["config_lr"] = vitc["config_lr"].astype(float)  # Ensure learning rate is float type
vitc = vitc[vitc["state"] == "finished"]  # Filter for finished runs

vit_base = df[df['id'] == "2fupnc3h"] # Base model
vit_conv_stem_only = df[df['id'] == "uaryeifu"]  # Conv stem only model
vitc_base = df[df['id'] == "n8dqbqan"]  # Conv stem only model
#vitc = pd.concat([vit_base, vit_conv_stem_only, vitc, vitc_base], ignore_index=True)  # Combine base and conv stem only models

vitc.reset_index(drop=True, inplace=True)  # Reset index after filtering
print(f"Number of runs in layerwise_freezing group: {len(vitc)}")

#Renaming
vitc = pu.rename(vitc).sort_values(by=val_accuracy, ascending=False)
vitc = pu.rename_adapter_factor_experiments(vitc)  # Rename columns for adapter factor experiments
vitc.rename(columns={
    "config_lora_rank": "Lora Rank",
    "config_trainable_parameters": trainable_params,
    "summary__runtime": time,
}, inplace=True)

vitc.replace({
    "Model": {
        "vitc_b_16_B=2_pretrained_PEFT_lora": "ViTc LoRA",
        "vitc_b_16_B=2_pretrained_PEFT_dora": "ViTc DoRA",
        "vitc_b_16_B=2_pretrained_PEFT_lora_eva": "ViTc LoRA (EVA)",
        "vitc_b_16_B=2_pretrained_PEFT_dora_eva": "ViTc DoRA (EVA)",
        "vitc_b_16_B=2_pretrained_PEFT_randlora": "ViTc RandLoRA",
        "vitc_b_16_B=2_pretrained_PEFT_shira": "ViTc SHiRA",


        "vitc_b_16_B=2_pretrained_only-conv-stem": "ViTc - conv stem only",
        "vit_b_16_B=2_pretrained": "ViT",
        "vit_b_16_B=2_pretrained_conv-stem": "ViTc"
    }}, inplace=True)  # Rename specific model names for clarity

vitc[time] = vitc[time].apply(lambda x: x/(60*60))  # Format runtime to 2 decimal places
# Some stats
for name in vitc["Model"].unique(): print(name)  # Print each model name
display(vitc)


/tmp/ipykernel_3591170/3406621312.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_3591170/3406621312.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Number of runs in layerwise_freezing group: 69
ViTc LoRA
ViTc LoRA (EVA)
ViTc SHiRA
ViTc DoRA (EVA)
ViTc DoRA
ViTc RandLoRA


id            Model     state        entity  \
15  0g71pg9q        ViTc LoRA  finished  raphaelmaser   
29  szc1yymj  ViTc LoRA (EVA)  finished  raphaelmaser   
3   qc2uey11       ViTc SHiRA  finished  raphaelmaser   
2   r4azecsz       ViTc SHiRA  finished  raphaelmaser   
32  2g22y6ls  ViTc LoRA (EVA)  finished  raphaelmaser   
30  rb60mlgc        ViTc LoRA  finished  raphaelmaser   
50  3iqp4zgf       ViTc SHiRA  finished  raphaelmaser   
6   x00kw09p       ViTc SHiRA  finished  raphaelmaser   
7   rmziu6x3       ViTc SHiRA  finished  raphaelmaser   
52  rchdfvmy        ViTc LoRA  finished  raphaelmaser   
19  3dbpx167  ViTc LoRA (EVA)  finished  raphaelmaser   
51  yddi2bxm  ViTc LoRA (EVA)  finished  raphaelmaser   
13  j0ejar3f        ViTc LoRA  finished  raphaelmaser   
21  tp5pcwwb  ViTc DoRA (EVA)  finished  raphaelmaser   
61  c13cuw75       ViTc SHiRA  finished  raphaelmaser   
27  om4nuv5s        ViTc DoRA  finished  raphaelmaser   
56  49fgj7sd  ViTc DoRA (EVA)  finished  raphaelmaser   
45  s9yv99im  ViTc LoRA (EVA)  finished  raphaelmaser   
67  8b7ulc47       ViTc SHiRA  finished  raphaelmaser   
39  ovkx8twd        ViTc DoRA  finished  raphaelmaser   
9   4sienj41        ViTc DoRA  finished  raphaelmaser   
14  l7u57s9a  ViTc DoRA (EVA)  finished  raphaelmaser   
18  84rnv1or  ViTc DoRA (EVA)  finished  raphaelmaser   
1   7dgtbtuu       ViTc SHiRA  finished  raphaelmaser   
57  o4pxiqgu  ViTc DoRA (EVA)  finished  raphaelmaser   
11  gct9lioy        ViTc DoRA  finished  raphaelmaser   
37  3tim8xlt        ViTc DoRA  finished  raphaelmaser   
49  33duxee8        ViTc DoRA  finished  raphaelmaser   
47  8sbckdnc  ViTc LoRA (EVA)  finished  raphaelmaser   
59  mkpklpir  ViTc DoRA (EVA)  finished  raphaelmaser   
0   oq2v77ht        ViTc DoRA  finished  raphaelmaser   
17  74a5ozqo  ViTc DoRA (EVA)  finished  raphaelmaser   
55  r8dk7bpq  ViTc DoRA (EVA)  finished  raphaelmaser   
43  t8u2ehji        ViTc DoRA  finished  raphaelmaser   
16  ud4lq8q9       ViTc SHiRA  finished  raphaelmaser   
24  n21bs7aw        ViTc LoRA  finished  raphaelmaser   
23  joxiwlq7  ViTc DoRA (EVA)  finished  raphaelmaser   
68  dkt4y0xg        ViTc DoRA  finished  raphaelmaser   
34  miqwxcix    ViTc RandLoRA  finished  raphaelmaser   
46  kl8nph8n       ViTc SHiRA  finished  raphaelmaser   
60  jz23ob3s        ViTc DoRA  finished  raphaelmaser   
26  uufv48z9       ViTc SHiRA  finished  raphaelmaser   
62  wy0km01u    ViTc RandLoRA  finished  raphaelmaser   
20  1xir2336    ViTc RandLoRA  finished  raphaelmaser   
53  j18jxeqe       ViTc SHiRA  finished  raphaelmaser   
48  8dh3ekca    ViTc RandLoRA  finished  raphaelmaser   
63  k2iau7l1  ViTc DoRA (EVA)  finished  raphaelmaser   
42  vzdak5yn    ViTc RandLoRA  finished  raphaelmaser   
33  zd2cmz19       ViTc SHiRA  finished  raphaelmaser   
12  vut5wb65    ViTc RandLoRA  finished  raphaelmaser   
5   c9lfk12q    ViTc RandLoRA  finished  raphaelmaser   
8   geqnvmgl    ViTc RandLoRA  finished  raphaelmaser   
22  3n26u1t1       ViTc SHiRA  finished  raphaelmaser   
28  ru1g9cit    ViTc RandLoRA  finished  raphaelmaser   
25  o2stcvc2    ViTc RandLoRA  finished  raphaelmaser   
66  9u5ged2b       ViTc SHiRA  finished  raphaelmaser   
35  zn2pi95s    ViTc RandLoRA  finished  raphaelmaser   
41  sf1ygoyz    ViTc RandLoRA  finished  raphaelmaser   
44  7uyiaono    ViTc RandLoRA  finished  raphaelmaser   
31  ndqzamb4    ViTc RandLoRA  finished  raphaelmaser   
4   bcotsy1e  ViTc LoRA (EVA)  finished  raphaelmaser   
10  7uz4hr5t        ViTc LoRA  finished  raphaelmaser   
36  33zu8wwe        ViTc LoRA  finished  raphaelmaser   
40  wzy9qhp0        ViTc LoRA  finished  raphaelmaser   
58  n2e9m5cf  ViTc LoRA (EVA)  finished  raphaelmaser   
38  nezoefn0        ViTc LoRA  finished  raphaelmaser   
54  td38vn23        ViTc LoRA  finished  raphaelmaser   
64  1yum36vq  ViTc LoRA (EVA)  finished  raphaelmaser   
65  qwlu4n2a  ViTc LoRA (EVA)  finished  raphaelmaser   

                   

In [33]:
# vitc = pd.concat([vit_c, vitc], ignore_index=True)  # Add the qkv LoRA factors to the DataFrame

## LR = 1e-3

In [34]:
high_lr = vitc[vitc["config_lr"] == 0.001]  # Filter for runs with high learning rate
generate_scatter_plot_accuracy(
    data=high_lr,
    x=trainable_params,
    y=val_accuracy,
    color="Model",
    scale_factor=2,
    title="ViTc - LR=1e-3",
    connect_dots=True,
    scale_dots_by_trainable_params=trainable_params
)

In [ ]:
generate_scatter_plot_accuracy(
    data=high_lr,
    x=trainable_params,
    y=grid_score,
    color="Model",
    scale_factor=2,
    title="ViTc - LR=1e-3",
    connect_dots=True,
    scale_dots_by_trainable_params=trainable_params
)

## LR=5e-4

In [40]:
low_lr = vitc[vitc["config_lr"] == 0.0005]  # Filter for runs with high learning rate
generate_scatter_plot_accuracy(
    data=low_lr,
    x=trainable_params,
    y=val_accuracy,
    color="Model",
    scale_factor=2,
    title="ViTc - LR=5e-4",
    connect_dots=True,
    scale_dots_by_trainable_params=trainable_params
)

In [41]:
generate_scatter_plot_accuracy(
    data=low_lr,
    x=trainable_params,
    y=grid_score,
    color="Model",
    scale_factor=2,
    title="ViTc - LR=5e-4",
    connect_dots=True,
    scale_dots_by_trainable_params=trainable_params
)